In [1]:
import pandas as pd
import time 
#import ast
from all_functions import *
import math
import requests
from pylatex import Document, Section, Subsection, Command
from pylatex.utils import italic, NoEscape
from selenium import webdriver
import os
import subprocess
import urllib.request


import sys

In [2]:
scrap_params = {
    'results_per_site' : 10,
    'job_title' : 'python',
    'city' : 'worcester',
    'site_list' : ['stack', 'indeed', 'linkedin'],
    'linked_in_username' : 'YOUR USERNAME',
    'linked_in_password' : 'YOUR PASSWORD',
    'chromedriver_location' : u'chromedriverlocation',
    'api_key' : 'YOURGOOGLEMAPSAPIKEY'
}
APIkey = scrap_params['api_key']

In [3]:
def get_address(query):
        base = 'https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input='
        locationbias= '42.3314584645106, -71.1039191294055'
        key = APIkey
        query = query
        fields = 'formatted_address,name,geometry'
        inputtype = 'textquery'
        
        url_search = base + query + '&inputtype=' + inputtype + '&fields=' + fields + '&locationbias=circle:2000@' + locationbias + '&key=' + key
        my_dict = requests.get(url_search).json()
        return my_dict

In [4]:
def get_commute(dest, origin):
    
    base = 'https://maps.googleapis.com/maps/api/distancematrix/json?units=imperial&'
    origin = 'origins=' + origin + '&'
    dest_str = 'destinations=' + dest + '&'
    api_key = 'key=' + APIkey
    
    url_search = base + origin + dest_str + api_key
    my_dict = requests.get(url_search).json()
    return my_dict

In [5]:
def dist_duration(company_list, id_list, api_key):
    comm_df = pd.DataFrame(columns=['company', 'place', 'address', 'distance', 'duration'])
    for index, company in enumerate(company_list):
        comm_dict = {}
        company = company
        tmp_id = id_list[int(index)]
        text_dict = get_address(str(company))
        if text_dict['status'] == 'ZERO_RESULTS':
            print(company + ' has 0 results in get_address.')
            base = 'https://maps.googleapis.com/maps/api/staticmap?'
            place = 'None'
            address = 'None'
            distance = 'None'
            duration = 'None'
            #params = {'center' : '8 Oswald Street, Boston MA 02120' , 'zoom' : '20', 'size' : '200x200' , 'maptype' : 'roadmap', 'key': api_key}
            params = {'center' : '8 Oswald Street, Boston MA 02120' , 'zoom' : '10', 'size' : '200x200' , 'maptype' : 'roadmap', 'key': api_key}
            map_url = requests.get(base, params).url
        else:
            address = text_dict['candidates'][0]['formatted_address']
            place = text_dict['candidates'][0]['name']    
            res_lat_lng = str(text_dict['candidates'][0]['geometry']['location']['lat']) + ',' + str(text_dict['candidates'][0]['geometry']['location']['lng'])
            base = 'https://maps.googleapis.com/maps/api/staticmap?'
            #params = {'center' : '8 Oswald Street, Boston MA 02120' , 'zoom' : '18', 'size' : '200x200' , 'maptype' : 'roadmap', 'markers': 'color:blue|label:S|' + res_lat_lng, 'key': api_key} 
            params = {'center' : '8 Oswald Street, Boston MA 02120' , 'size' : '200x200' , 'maptype' : 'roadmap', 'markers': 'color:blue|label:S|' + res_lat_lng, 'key': api_key} 
            map_url = requests.get(base, params).url
            
            
            comm = get_commute(dest=res_lat_lng, origin='8 Oswald Street')
            if comm['rows'][0]['elements'][0]['status'] == 'ZERO_RESULTS':
                print(company + ' has 0 results in get_commute.')
                continue
            else:
                try:
                    distance = (comm['rows'][0]['elements'][0]['distance']['text'])
                    duration = (comm['rows'][0]['elements'][0]['duration']['text'])
                except:
                    #print(comm)
                    break

        comm_dict['company'] = company
        comm_dict['place'] = place
        comm_dict['address'] = address
        comm_dict['distance'] = distance
        comm_dict['duration'] = duration
        comm_dict['sID'] = tmp_id
        comm_dict['map_url'] = map_url
        comm_tmp_df = pd.DataFrame(comm_dict, index = [comm_dict['sID']])
        comm_df = comm_df.append(comm_tmp_df, sort=False)
    return comm_df

In [39]:
def ult_scrap(**scrap_params):
    results = scrap_params['results_per_site']
    job_title = scrap_params['job_title']
    city = scrap_params['city'] 
    site_list = scrap_params['site_list']
    linked_in_username = scrap_params['linked_in_username'] 
    linked_in_password = scrap_params['linked_in_password']
    chromedriver_location = scrap_params['chromedriver_location'] 
    api_key = scrap_params['api_key'] 
    
    file = 1
    SKIPPER = 0 # Skips this many of the first jobs
    #job_col_list = ['site', 'sID',  'title', 'company', 'location', 'salary', 'summary', 'tags','id', 'comp_loc', 'link', 'pulldate','postdate', 'ezapply']
    #job_df = pd.DataFrame(columns=job_col_list).set_index('sID')
    indeed_df = pd.DataFrame()
    linkedin_df = pd.DataFrame()
    stack_df = pd.DataFrame()

    num = 0
    options = Options()
    options.add_argument("--headless") # Runs Chrome in headless mode.
    options.add_argument('--no-sandbox') # Bypass OS security model
    options.add_argument('--disable-gpu')  # applicable to windows os only
    options.add_argument('start-maximized') # 
    options.add_argument('disable-infobars')
    options.add_argument("--disable-extensions")
    wd = webdriver.Chrome(chromedriver_location, options=options)
    for site in site_list: 
        site_ind = results
        
        
        if site == 'stack':
            selen = True
            
            if selen == True:
                list_o_divs = []
                #tot_pg = ((results // 10) + 1)
                res_left = results
                tot_pg = ((results // 25) + 1)
                print('stack total pages: ' + str(tot_pg))
                #print('indeed total pages: ' + str(tot_pg))
                for i in range(tot_pg):
                    url = 'https://stackoverflow.com/jobs?q=python&sort=p&l=boston&d=20&u=Miles&start=' + str(i*25)
                    #url = 'https://www.indeed.com/jobs?q=python&l=worcester&start=0&sort=date&radius=25'
                    print(url)
                    wd.get(url)
                    #################### search results ####################     
                    companies = [company.text for company in wd.find_elements_by_xpath("//div[@class='fc-black-700 fs-body2 -company']")]
                    
                    res_len = min(len(companies), int(res_left))
                    
                    #l_companies = [company.text.lower().split(',')[0] for company in wd.find_elements_by_xpath("//div[@data-tn-component='organicJob']//span[@class='company']")]
                    titles = [title.text for title in wd.find_elements_by_xpath("//h2[@class='fs-subheading job-details__spaced mb4']")]
                    tags = [tag.text for tag in wd.find_elements_by_xpath("//div[@class='mt12 -tag']")]
                    perks = [perk.text for perk in wd.find_elements_by_xpath("//div[@class='mt2 -perks']")]
                    postdates = [date.text for date in wd.find_elements_by_xpath("//span[contains(@class, 'ps-absolute pt2 r0 fc-black-500 fs-body1 pr12')]")]
                    #description = [description.text for description in wd.find_elements_by_xpath("//div[@data-tn-component='organicJob']//span[@class='summary']")]
                    post_links = [title.get_attribute('href') for title in wd.find_elements_by_xpath("//h2[@class='fs-subheading job-details__spaced mb4']//a")]
                    #comp_links = ['https://www.indeed.com/cmp/' + str(company).replace(' ', '-') for company in companies]                    

                    
                    if res_len < 25:
                        companies = companies[:res_len]
                        titles = titles[:res_len]
                        tags = tags[:res_len]
                        perks = perks[:res_len]
                        post_links = post_links[:res_len]
                        postdates = postdates[:res_len]
                    
                    sID = [('s-ID-' + str(n)) for n in range(num, num + res_len)]
                    pulldates = [str(time.strftime("%m%d%Y", time.localtime(time.time()))) for n in range(num, num + res_len)]
                    
                    print(len(titles), len(companies), len(post_links), len(postdates), len(pulldates))
                    #return 'bitch'
                    stack_df = pd.DataFrame({'ID' : sID, 'title' : titles, 'company' : companies, 'post_link' : post_links, 'postdate' : postdates, 'pulldate' : pulldates}) # functional
                    #return list_df
            if selen == False:
                site_ind = 0 # set to 0 at start of site and compared to results so there are 'results' number of pulls per site
                list_o_divs = [] # holds divs if multiple pages
                tmp_dict = {} # holds info on each div loop
                # https://stackoverflow.com/jobs?q=data+scientist&l=boston&d=20&u=Miles
                pg = 1 # page counter 
                base = 'https://stackoverflow.com/jobs?'
                tot_pg = ((results // 20) + 1)
                print('Stack overflow total pages: ' + str(tot_pg))
                while pg <= tot_pg: 
                    params = {'q' : job_title, 'sort' : 'p', 'l' : city, 'd' : '20', 'u': 'Miles', 'pg': pg} 
                    page = requests.get(base, params) 
                    print(page.url)
                    time.sleep(1)
                    soup = get_soup(page.text)
                    divs = soup.find_all(name="div", attrs={"class":"-job-summary"})
                    list_o_divs.append(divs)
                    pg = pg + 1
                for divs in list_o_divs:
                    for div in divs:
                        if (site_ind < results): # if there is a div past the number of results for the site, doesn't add it to tmp_dict
                            tmp_dict['sID'] = 'ID' + str(num)
                            tmp_dict['site'] = site
                            tmp_dict['pulldate'] = str(time.strftime("%m-%d-%Y_%I-%M_%p", time.localtime(time.time())))
                            tmp_dict['link'] = 'https://www.stackoverflow.com' + div.find(name='a', attrs={'class':'s-link'})['href']
                            # tmp_dict['link'] = extract_link(div, site)
                            tmp_dict['title'] = div.find(name='a', attrs={'class':'s-link'})['title']
                            # tmp_dict['title'] = extract_job_title(div, site)
                            tmp_dict['summary'] = 'No Stack Summary yet'
                            if div.find('div', attrs={'class': '-perks'}).text is None:
                                tmp_dict['salary'] = 'NA'
                            else:
                                div.find('div', attrs={'class': '-perks'}).text.replace('\n', ' ').replace('\r', '').strip()
                            # tmp_dict['salary'] = extract_salary(div, site)
                            tmp_dict['location'] = div.find_all('div', attrs={'class': '-company'}).split('-')[1].text
                            # tmp_dict['location'] = extract_location(div, site).strip()
                            tmp_dict['tags'] = ', '.join(div.find_all('a', attrs={'class': 'post-tag job-link no-tag-menu'}))
                            # tmp_dict['tags'] = extract_tags(div, site)
                            tmp_dict['id'] = div.find('span', attrs={'class': 'fav-toggle'})['data-jobid']
                            #tmp_dict['id'] = extract_id(div, site)
                            tmp_dict['ezapply'] = 'No Easy Applications on Stack'
                            if div.find('span', attrs={'class': 'ps-absolute pt2 r0 fc-black-500 fs-body1 pr12 t24'}).text is None:
                                tmp_dict['postdate'] = div.find('span', attrs={'class': 'ps-absolute pt2 r0 fc-black-500 fs-body1 pr12 t32'}).text
                            else:
                                tmp_dict['postdate'] = div.find('span', attrs={'class': 'ps-absolute pt2 r0 fc-black-500 fs-body1 pr12 t24'}).text
                            # tmp_dict['postdate'] = extract_postdate(div, site)
                            tmp_dict['company'] = div.find_all(name="div", attrs={"class":"-company"}).split('-')[0].text.replace('\n', '').replace('\r','')
                            #tmp_dict['company'] = extract_company(div, site).split('-')[0].strip()
                            tmp_dict['comp_loc'] = div.find_all('div', attrs={'class': '-company'}).text
                            tmp_df = pd.DataFrame(tmp_dict, index=[tmp_dict['sID']])
                            job_df = job_df.append(tmp_df, sort=False)
                            num = num + 1
                            site_ind = site_ind + 1
            
        if site == 'indeed':
            
            selen = True 
            
            if selen == True:
                list_o_divs = []
                #tot_pg = ((results // 10) + 1)
                res_left = results
                tot_pg = ((results // 10) + 1)
                print('indeed total pages: ' + str(tot_pg))
                #print('indeed total pages: ' + str(tot_pg))
                for i in range(tot_pg):
                    url = 'https://www.indeed.com/jobs?q=' + job_title + '&l=' + city + '&start=' + str(i*10) + '&sort=' + 'date' + '&radius=' + '25'
                    #url = 'https://www.indeed.com/jobs?q=python&l=worcester&start=0&sort=date&radius=25'
                    print(url)
                    wd.get(url)
                    #################### search results ####################     
                    companies = [company.text for company in wd.find_elements_by_xpath("//div[@data-tn-component='organicJob']//span[@class='company']")]
                    
                    res_len = min(len(companies), int(res_left))
                    
                    l_companies = [company.text.lower().split(',')[0] for company in wd.find_elements_by_xpath("//div[@data-tn-component='organicJob']//span[@class='company']")]
                    titles = [title.text for title in wd.find_elements_by_xpath("//div[@data-tn-component='organicJob']//h2[@class='jobtitle']")]
                    description = [description.text for description in wd.find_elements_by_xpath("//div[@data-tn-component='organicJob']//span[@class='summary']")]
                    post_links = [post_link.get_attribute('href') for post_link in wd.find_elements_by_xpath("//div[@data-tn-component='organicJob']//h2[@class='jobtitle']//a")]
                    comp_links = ['https://www.indeed.com/cmp/' + str(company).replace(' ', '-') for company in companies]                    
                    
                    if res_len < 25:
                        l_companies = l_companies[:res_len]
                        titles = titles[:res_len]
                        description = description[:res_len]
                        companies = companies[:res_len]
                        post_links = post_links[:res_len]
                        comp_links = comp_links[:res_len]
                    
                    sID = [('i-ID-' + str(n)) for n in range(num, num + res_len)]
                    pulldates = [str(time.strftime("%m%d%Y", time.localtime(time.time()))) for n in range(num, num + res_len)]
                    
                    list_df = pd.DataFrame({'ID' : sID, 'title' : titles, 'company' : companies, 'summary' : description, 'post_link' : post_links, 'comp_link' : comp_links, 'l_company' : l_companies, 'pulldate' : pulldates}) # functional
                    list_df['ID'] = list_df['ID'].astype('str')
                    
                    #################### comp list ####################     
                    comp_list = []
                    for index, link in enumerate(comp_links):
                        index = sID[index]
                        wd.get(link)
                        try:
                            company_name = wd.find_element_by_xpath("//div[@class='cmp-company-name-container']").text
                        except:
                            comp_list.append(['No company name found', '', '', '', '', ''])
                            continue
                        l_company_name = wd.find_element_by_xpath("//div[@class='cmp-company-name-container']").text.lower().split(',')[0]
                        try:
                            average_comp_rating = wd.find_element_by_xpath("//span[@class='cmp-header-rating-average']").text
                        except:
                            average_comp_rating = 'Not ranked.'
                        salaries_link = 'https://www.indeed.com/cmp/' + company_name + '/salaries/'
                        reviews_link = 'https://www.indeed.com/cmp/' + company_name + '/reviews/'
                        comp_list.append([str(index), company_name, average_comp_rating, salaries_link, reviews_link, l_company_name])
                    comp_df = pd.DataFrame(comp_list, columns=['ID', 'company', 'rating', 'salaries_link', 'reviews_link', 'l_company'])

                    #################### Post list ####################     
                    post_list = []
                    for index, link in enumerate(post_links):
                        index = sID[index]
                        wd.get(link)
                        jobtitle = wd.find_element_by_xpath("//h3[contains(@class, 'jobsearch-JobInfoHeader-title')]").text
                        try:
                            company_pic = wd.find_element_by_xpath("//img[@class='jobsearch-CompanyAvatar-image']").get_attribute('src')
                        except:
                            company_pic = 'NA'
                        line = wd.find_element_by_xpath("//div[contains(@class, 'jobsearch-InlineCompanyRating')]").text
                        location = line.split('\n')[-1]
                        company_name = line.split('\n')[0]
                        l_company_name = company_name.lower().split(',')[0]
                        comp_loc = company_name + ', ' + location
                        body = wd.find_element_by_xpath("//div[contains(@class, 'jobsearch-JobComponent-description')]").text
                        post_time = wd.find_element_by_xpath("//div[@class='jobsearch-JobMetadataFooter']").text.split(' - ')[1].strip()
                        post_list.append([str(index), jobtitle, company_name, comp_loc, post_time, location, company_pic, body, l_company_name])
                        
                    post_df = pd.DataFrame(post_list, columns=['ID', 'title', 'company', 'comp_loc', 'postdate', 'location', 'company_pic', 'body_text', 'l_company'])
                    
                    #################### DF merging ####################                    
                    post_comp_df = comp_df.merge(post_df, on=['ID'], how='right').drop(columns=['l_company_y', 'company_y']).rename({'company_x' : 'company', 'l_company_x' : 'l_company'}, axis='columns')
                    post_comp_df['ID'] = post_comp_df['ID'].astype('str')
                    tmp_df = post_comp_df.merge(list_df, on=['ID'], how='outer').drop(columns=['l_company_y', 'company_y', 'title_y']).rename({'company_x' : 'company', 'l_company_x' : 'l_company', 'title_x' : 'title'}, axis='columns')
                    indeed_df = indeed_df.append(tmp_df, sort=False)
                    
                    num += len(tmp_df)
                    res_left = res_left - len(tmp_df)
            if selen == False:
                    pass
                    # code is below             
                    
                    
        if site == 'linkedin':
            
            tot_pg = ((results // 25) + 1)
            res_left = results
            print('linkedin total pages: ' + str(tot_pg))
            login_url = 'https://www.linkedin.com/'
            wd.get(login_url)
            #wd.find_element_by_xpath("//a[@class='sign-in-link']").click
            #wd.find_element_by_xpath("//a[@class='form-toggle']").click
            login_email = wd.find_element_by_id("login-email")
            login_email.send_keys(linked_in_username)
            login_pass = wd.find_element_by_id("login-password")
            login_pass.send_keys(linked_in_password)
            wd.find_element_by_id("login-submit").click
            #cookies = wd.get_cookie('bcookie')
            
            for i in range(tot_pg):
                url = 'http://www.linkedin.com/jobs/search/?keywords=data+science&location=Boston&sortBy=DD&start=' + str(i*25)
                print('list url : ', url)
                wd.get(url)
                
                listings = wd.find_elements_by_xpath("//li[@class='job-listing']")
                res_len = min(len(listings), int(res_left))
                
                post_link_base = 'https://www.linkedin.com/uas/login?session_redirect='
                post_links = [l.get_attribute('href') for l in wd.find_elements_by_xpath("//a[@class='job-title-link']")]
                comp_links = []
                for elem in wd.find_elements_by_xpath("//div[@class='company-name']"):
                    if elem.find_element_by_xpath("//a") == None:
                        comp_links.append('NA')
                    else:
                        comp_links.append(elem.find_element_by_xpath("//a").get_attribute('href'))
                #comp_links = [l.get_attribute('href') for l in wd.find_elements_by_xpath("//a[@class='company-name-link']")]
                titles = [t.text for t in wd.find_elements_by_xpath("//span[@class='job-title-text']")]
                locations = [L.text for L in wd.find_elements_by_xpath("//span[@class='job-location']/span")]
                companies = [c.text for c in wd.find_elements_by_xpath("//span[@class='company-name-text']")]
                descr = [d.text for d in wd.find_elements_by_xpath("//div[@class='job-description']")]
                ezapply = [e.text for e in wd.find_elements_by_xpath("//div[@class='job-flavor-in-apply-container']")]
                
                company_pics = [i.get_attribute('src') for i in wd.find_elements_by_xpath("//a[@class='company-logo-link']//img")]
                
                
                if res_len < 25:
                    post_links = post_links[:res_len]
                    comp_links = comp_links[:res_len]
                    titles = titles[:res_len]
                    locations = locations[:res_len]
                    companies = companies[:res_len]
                    descr = descr[:res_len]
                    ezapply = ezapply[:res_len]
                    company_pics = company_pics[:res_len]
                sID = [('L-ID-' + str(n)) for n in range(results-res_left, res_left)]
                pulldates = [str(time.strftime("%m%d%Y", time.localtime(time.time()))) for n in range(num, num + res_len)]
                comp_loc = []
                for i, elem in enumerate(companies):
                    comp_loc_str = (elem.strip() + ', ' + locations[i].strip())
                    comp_loc.append(comp_loc_str)
                try:
                    tmp_df = pd.DataFrame({'ID': sID, 'title' : titles, 'company' : companies, 'location' : locations, 
                                       'post_link' : post_links, 'comp_link' : comp_links, 'ezapply' : ezapply, 'company_pic' : company_pics, 
                                       'pulldate' : pulldates, 'comp_loc': comp_loc})
                except:
                    print(len(titles), len(comp_loc), len(companies), len(locations), len(post_links), len(comp_links), len(ezapply), len(company_pics), len(pulldates))
                    return 'bitch'

                
                #list_df = pd.DataFrame({'index' : sID, 'title' : titles, 'company' : companies, 'summary' : description, 'post_links' : post_links, 'comp_links' : comp_links, 'l_company' : l_companies}) # functional
                res_left = res_left - len(tmp_df)
                
                                    #################### DF merging ####################
                    
                #post_comp_df = comp_df.merge(post_df, on=['index'], how='right').drop(columns=['l_company_y', 'company_y']).rename({'company_x' : 'company', 'l_company_x' : 'l_company'}, axis='columns')
                #post_comp_df['index'] = post_comp_df['index'].astype('str')
                #merge_df = post_df.merge(tmp_df, on=['index'], how='outer').drop(columns=['l_company_y', 'company_y', 'title_y']).rename({'company_x' : 'company', 'l_company_x' : 'l_company', 'title_x' : 'title'}, axis='columns')
                linkedin_df = linkedin_df.append(tmp_df, sort=False)
    
    job_df = linkedin_df.append(indeed_df, sort=False)
    job_df = job_df.append(stack_df, sort=False)
    #companies = job_df['comp_loc']
    #ids = job_df.index.tolist()
    
    
    
    #comm_df = dist_duration(companies, ids, api_key)

    #fin_df = job_df.merge(comm_df, how = 'left', left_index=True, right_index=True)
    clean_df = False
    
    if clean_df == True:
        #fin_df.rename({'company_x' : 'company'}, axis='columns', inplace=True)
        #fin_df.drop(['sID_x', 'sID_y', 'company_y'], axis='columns', inplace=True)
        #nice_col_list = ['site', 'title', 'comp_loc', 'distance', 'duration', 'summary', 'tags', 'link', 'ezapply']
        #nice_df = fin_df[nice_col_list]
        
        return clean_df
    file = file + 1
    wd.quit()
    output_dict = {'job_title' : job_title, 'city' : city}
    return job_df


In [40]:
fetch = ult_scrap(**{
    'results_per_site' : 15,
    'job_title' : 'python',
    'city' : 'boston',
    'site_list' : ['linkedin', 'stack', 'indeed'],
    'linked_in_username' : 'YOURUSERNAMEHERE',
    'linked_in_password' : 'YOURPASSWORD',
    'chromedriver_location' : u'YOURCHROMEDRIVIERLOC',
    'api_key' : 'YOURGOOGLEMAPSAPIKEY'
})
fetch

linkedin total pages: 1
list url :  http://www.linkedin.com/jobs/search/?keywords=data+science&location=Boston&sortBy=DD&start=0
stack total pages: 1
https://stackoverflow.com/jobs?q=python&sort=p&l=boston&d=20&u=Miles&start=0
15 15 15 15 15
indeed total pages: 2
https://www.indeed.com/jobs?q=python&l=boston&start=0&sort=date&radius=25
https://www.indeed.com/jobs?q=python&l=boston&start=10&sort=date&radius=25


,ID,title,company,location,post_link,comp_link,ezapply,company_pic,pulldate,comp_loc,rating,salaries_link,reviews_link,l_company,postdate,body_text,summary
0,L-ID-0,Technical Delivery Manager,Appian,"Boston, MA, US",https://www.linkedin.com/jobs/view/technical-d...,https://www.linkedin.com/jobs?trk=jobs_chrome_...,,https://www.linkedin.com/scds/common/u/images/...,11082018,"Appian, Boston, MA, US",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,L-ID-1,Sr UI Angular 2 Developer - US Citizen/Green c...,Financial firm,"Boston, MA, US",https://www.linkedin.com/jobs/view/sr-ui-angul...,https://www.linkedin.com/jobs?trk=jobs_chrome_...,Easy Apply,https://www.linkedin.com/scds/common/u/images/...,11082018,"Financial firm, Boston, MA, US",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,L-ID-2,VP of Product Management,Quest Groups,Greater Boston Area,https://www.linkedin.com/jobs/view/vp-of-produ...,https://www.linkedin.com/jobs?trk=jobs_chrome_...,Easy Apply,https://www.linkedin.com/scds/common/u/images/...,11082018,"Quest Groups, Greater Boston Area",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,L-ID-3,"Technical Associate I, AgeLab - Cambridge",Massachusetts Institute of Technology,"Cambridge, MA, US",https://www.linkedin.com/jobs/view/technical-a...,https://www.linkedin.com/jobs?trk=jobs_chrome_...,,https://media.licdn.com/dms/image/C4E0BAQHBK7q...,11082018,"Massachusetts Institute of Technology, Cambrid...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,L-ID-4,Product Manager -,"Casenet, LLC","Bedford, MA, US",https://www.linkedin.com/jobs/view/product-man...,https://www.linkedin.com/jobs?trk=jobs_chrome_...,,None,11082018,"Casenet, LLC, Bedford, MA, US",NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,L-ID-5,Engineer l,Weston & Sampson,"Foxborough, MA, US",https://www.linkedin.com/jobs/view/engineer-l-...,https://www.linkedin.com/jobs?trk=jobs_chrome_...,,https://media.licdn.com/dms/image/C4D0BAQFbppI...,11082018,"Weston & Sampson, Foxborough, MA, US",NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,L-ID-6,Senior Tech Lead,Trianz,"Manchester, NH, US",https://www.linkedin.com/jobs/view/senior-tech...,https://www.linkedin.com/jobs?trk=jobs_chrome_...,,None,11082018,"Trianz, Manchester, NH, US",NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,L-ID-7,Healthcare Data Engineer,"iSpecimen, Inc.","Boston, MA, US",https://www.linkedin.com/jobs/view/healthcare-...,https://www.linkedin.com/jobs?trk=jobs_chrome_...,,None,11082018,"iSpecimen, Inc., Boston, MA, US",NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,L-ID-8,QA Automation Engineer w/ Innovative Marketing...,Workbridge Associates,"Boston, MA, US",https://www.linkedin.com/jobs/view/qa-automati...,https://www.linkedin.com/jobs?trk=jobs_chrome_...,,None,11082018,"Workbridge Associates, Boston, MA, US",NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,L-ID-9,Senior Sales Operations Planning Analyst* - Fa...,Acushnet Company,"Fairhaven, MA, US",https://www.linkedin.com/jobs/view/senior-sale...,https://www.linkedin.com/jobs?trk=jobs_chrome_...,,https://media.licdn.com/dms/image/C4E0BAQE8M_T...,11082018,"Acushnet Company, Fairhaven, MA, US",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [180]:
'''for elem in fetch['post_link']:
    print(elem)'''

"for elem in fetch['post_link']:\n    print(elem)"

In [120]:
options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument('--no-sandbox')
options.add_argument("--disable-extensions")
options.add_argument('--log-level=3')
wd = webdriver.Chrome(scrap_params['chromedriver_location'], options=options)
url = 'https://www.indeed.com/jobs?q=python&l=worcester&start=0&sort=date&radius=25'
wd.get(url)


In [12]:
options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument('--no-sandbox')
options.add_argument("--disable-extensions")
options.add_argument('--log-level=3')
wd = webdriver.Chrome(scrap_params['chromedriver_location'], options=options)
url = 'https://www.indeed.com/jobs?q=python&l=worcester&start=0&sort=date&radius=25'
print(url)
wd.get(url)
companies = []

#companies = [company.text for company in wd.find_elements_by_xpath("//div[@data-tn-component='organicJob']")]
companies = [company.text for company in wd.find_elements_by_xpath("//div[@data-tn-component='organicJob']//span[@class='company']")]
titles = [title.text for title in wd.find_elements_by_xpath("//div[@data-tn-component='organicJob']//h2[@class='jobtitle']")]
description = [description.text for description in wd.find_elements_by_xpath("//div[@data-tn-component='organicJob']//span[@class='summary']")]
post_links = [post_link.get_attribute('href') for post_link in wd.find_elements_by_xpath("//div[@data-tn-component='organicJob']//h2[@class='jobtitle']//a")]
comp_links = ['https://www.indeed.com/cmp/' + str(company).replace(' ', '-') for company in companies]

tmp_df = pd.DataFrame({'sID' : 'sID', 'site' : 'indeed', 'title' : titles, 'company' : companies, 'summary' : description, 'post_links' : post_links, 'comp_links' : comp_links})

comp_list = []
for link in comp_links:
    wd.get(link)
    company_name = wd.find_element_by_xpath("//div[@class='cmp-company-name-container']").text
    try:
        average_comp_rating = wd.find_element_by_xpath("//span[@class='cmp-header-rating-average']").text
    except:
        average_comp_rating = 'Not ranked.'
    salaries_link = 'https://www.indeed.com/cmp/' + company_name + '/salaries/'
    reviews_link = 'https://www.indeed.com/cmp/' + company_name + '/reviews/'
    comp_list.append([company_name, average_comp_rating, salaries_link, reviews_link])
comp_df = pd.DataFrame(comp_list, columns=['company', 'rating', 'salaries_link', 'reviews_link'])

post_list = []
for link in post_links:
    wd.get(link)
    jobtitle = wd.find_element_by_xpath("//h3[contains(@class, 'jobsearch-JobInfoHeader-title')]").text
    #jobtitle = wd.find_element_by_xpath("//h3[@class='jobsearch-JobInfoHeader-title']").text
    #company_name = wd.find_element_by_xpath("//div[@class='icl-u-lg-mr--sm icl-u-xs-mr--xs']").text
    try:
        company_pic = wd.find_element_by_xpath("//img[@class='jobsearch-CompanyAvatar-image']").get_attribute('src')
    except:
        company_pic = 'NA'
    line = wd.find_element_by_xpath("//div[contains(@class, 'jobsearch-InlineCompanyRating')]").text
    location = line.split('\n')[2]
    company_name = line.split('\n')[0]
    body = wd.find_element_by_xpath("//div[contains(@class, 'jobsearch-JobComponent-description')]").text
    post_time = wd.find_element_by_xpath("//div[@class='jobsearch-JobMetadataFooter']").text.split('-')[1].strip()
    post_list.append([jobtitle, company_name, post_time, location, company_pic, body])
post_df = pd.DataFrame(post_list, columns=['title', 'company', 'posttime', 'location', 'company_pic', 'body_text'])
job_df = comp_df.merge(post_df, on='company', how='left')
job_df = job_df.merge(tmp_df, on=['company', 'title'], how='left')
wd.quit()

https://www.indeed.com/jobs?q=python&l=worcester&start=0&sort=date&radius=25
10 
 10 
 10 
 10 
 10
-----------------------------------------------------------------------------
-----------------------------------------------------------------------------
                        company       rating  \
0                  Diverse Lynx          4.8   
1                   PerkinElmer          3.8   
2                       Marvell          3.4   
3                       Marvell          3.4   
4                          Bose          4.2   
5                          Bose          4.2   
6                          Bose          4.2   
7                          Bose          4.2   
8                       Marvell          3.4   
9                       Marvell          3.4   
10                         Dell          4.0   
11                        Intel          4.1   
12  The Hanover Insurance Group          3.8   
13           Spill Center, Inc.  Not ranked.   

                       

In [8]:
tmp_df['comp_links']

0              https://www.indeed.com/cmp/Diverse-Lynx
1               https://www.indeed.com/cmp/PerkinElmer
2                   https://www.indeed.com/cmp/Marvell
3                      https://www.indeed.com/cmp/Bose
4                      https://www.indeed.com/cmp/Bose
5                   https://www.indeed.com/cmp/Marvell
6                     https://www.indeed.com/cmp/Intel
7                      https://www.indeed.com/cmp/DELL
8    https://www.indeed.com/cmp/The-Hanover-Insuran...
9        https://www.indeed.com/cmp/Spill-Center,-Inc.
Name: comp_links, dtype: object

In [152]:
def tex_output(fetch, suffix=''):
    fetch_df = fetch
    #fetch_dict = fetch[1]
    map_list = []
    content = r'''\UseRawInputEncoding%
\documentclass{{article}}%
\usepackage[T1]{{fontenc}}%
\usepackage[utf8]{{inputenc}}%
\inputencoding{utf8}
\usepackage{{lmodern}}%
\usepackage{graphicx}
\usepackage{{textcomp}}%
\usepackage{{lastpage}}%

\begin{document}%
\normalsize%'''
    
    for index, row in fetch_df.iterrows():
        print(index)
        print(row['map_url'])
        map_file_name = str(index) + '.jpg'
        map_list.append(map_file_name)
        urllib.request.urlretrieve(str(row['map_url']), map_file_name)
        content = content + r'''
\section{{{jobtitle}}}%
\subsection{{{comploc}}}
{{{address}}}
{{{distance}}}
{{{duration}}}


\begin{{figure}}
\includegraphics{{{map_file_name}}}
\end{{figure}}

\subsection{{Salary}}%
{{{salary}}}

\subsection{{Link}}%
{{{link}}}

\newpage


'''.format(**{'jobtitle': row['title'], 'salary' : row['salary'], 'comploc' : row['comp_loc'], 'address' : row['address'], 'distance' : row['distance'], 
              'duration' : row['duration'], 'link' : row['link'], 'map_file_name' : map_file_name}).replace('&', '\&').replace('$', '\$').replace('#', '\#').replace('|', '').replace('_', '\_')        
        filename = 'job_scrap_output' + suffix + '.tex'

    content = content + '''
\end{document}%'''
    with open(filename,'w', encoding='utf-8') as f:
        f.write(content)
    cmd = ['pdflatex', filename]
    subprocess.check_output(cmd)

    os.unlink('job_scrap_output.log')
    os.unlink('job_scrap_output.aux')
    for file in map_list:
        os.unlink(file)

In [153]:
tex_output(fetch)

ID0


KeyError: 'map_url'

In [ ]:
### Here's below

'''indeed_pg_num = 0
                site_ind = 0 # set to 0 at start of site and compared to results so there are 'results' number of pulls per site
                list_o_divs = [] # holds divs if multiple pages
                tmp_dict = {} # holds info on each div loop
                # https://www.indeed.com/jobs?q=data+science&l=boston&sort=date
                res_left = results
                base = 'https://www.indeed.com/jobs?'
                while res_left > 0:            
                    params = {'q' : job_title,'l' : city, 'start' : (results-res_left),  'sort' : 'date', 'radius': '25'}
                    page = requests.get(base, params) 
                    time.sleep(1)  
                    soup = get_soup(page.text)
                    divs = soup.find_all(name="div", attrs={"class":"row"})
                    list_o_divs.append(divs)
                    res_left = res_left - len(divs)
                    indeed_pg_num = indeed_pg_num + 1
                print('total indeed pages : ' + str(indeed_pg_num))
                for divs in list_o_divs:
                    for div in divs:
                        if (site_ind < results):
                            tmp_dict['sID'] = 'ID' + str(num)
                            tmp_dict['site'] = site
                            tmp_dict['pulldate'] = str(time.strftime("%M%D%y", time.localtime(time.time())))
                            tmp_dict['link'] = 'http://www.indeed.com' + div.find(name='a', attrs={'class':'turnstileLink'})['href']
                            #tmp_dict['link'] = extract_link(div, site)
                            tmp_dict['title'] = div.find(name='a', attrs={'class':'turnstileLink'})['title']
                            # tmp_dict['title'] = extract_job_title(div, site)
                            tmp_dict['summary'] = div.find('span', attrs={'class': 'summary'}).text.strip()
                            # tmp_dict['summary'] = extract_summary(div, site)
                            #tmp_dict['salary'] = extract_salary(div, site)
                            # tmp_dict['location'] = extract_location(div, site).strip() # two things
                            if div.find('span', attrs={'class': 'location'}).text is None:
                                tmp_dict['location'] = div.find('div', attrs={'class': 'location'}).text.strip()
                            else:
                                tmp_dict['location'] = div.find('span', attrs={'class': 'location'}).text.strip()
                            tmp_dict['tags'] = extract_tags(div, site)
                            #tmp_dict['id'] = extract_id(div, site)
                            tmp_dict['id'] = div['id']
                            if div.find('div', attrs={'class': 'iaP'}) is not None:
                                tmp_dict['ezapply'] = True
                            else:
                                tmp_dict['ezapply'] = False
                            # tmp_dict['ezapply'] = extract_easyapply(div, site)
                            tmp_dict['postdate'] = extract_postdate(div, site)
                            #tmp_dict['company'] = extract_company(div, site).strip()
                            tmp_dict['company'] = div.find(name="span", attrs={"class":"company"}).text.replace('\n', '').strip()
                            tmp_dict['comp_loc'] = str(extract_company(div, site).strip() + ', ' + extract_location(div, site).strip())
                            tmp_df = pd.DataFrame(tmp_dict, index=[tmp_dict['sID']])
                            job_df = job_df.append(tmp_df, sort=False)
                            num = num + 1
                            site_ind = site_ind + 1'''